### Name : Gede Indra Adi Brata
### NIM  : 225150200111006

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification

# Tugas

Lakukan modifikasi kode untuk algoritma lain dari LVQ yaitu LVQ2, dan LVQ2.1

## LVQ

In [2]:
import numpy as np

class LVQ:
    
    def __init__(self):
        self.weights = np.array([]) 
        self.labels = np.array([])
    
    def train(self, training_data, target_labels, learning_rate, decay_factor, max_epochs):
        unique_labels, initial_indices = np.unique(target_labels, return_index=True)
        self.labels = unique_labels
        self.weights = training_data[initial_indices].astype(np.float64)
        remaining_data = np.array([sample for i, sample in enumerate(zip(training_data, target_labels)) if i not in initial_indices], dtype='object')
        training_data, target_labels = remaining_data[:, 0], remaining_data[:, 1]
        
        epoch = 0
        
        while epoch < max_epochs:
            for i, sample in enumerate(training_data):
                distances = [np.sum((weight - sample) ** 2) for weight in self.weights]
                closest_index = np.argmin(distances)
                adjustment_sign = 1 if target_labels[i] == unique_labels[closest_index] else -1
                self.weights[closest_index] += adjustment_sign * learning_rate * (sample - self.weights[closest_index])
            
            learning_rate *= decay_factor
            epoch += 1
        
            print(f'Weights: {self.weights}')
        
    def predict(self, input_data):
        centers, labels = self.weights, self.labels
        predictions = []
        for sample in input_data:
            distances = [np.sum((center - sample) ** 2) for center in centers]
            predictions.append(labels[np.argmin(distances)])
        return predictions

## LVQ 2

In [3]:
class LVQ2(LVQ):
    def train(self, training_data, target_labels, learning_rate, decay_factor, max_epochs, epsilon=0.3):
        unique_labels, initial_indices = np.unique(target_labels, return_index=True)
        self.labels = unique_labels
        self.weights = training_data[initial_indices].astype(np.float64)
        remaining_data = np.array([(sample, label) for i, (sample, label) in enumerate(zip(training_data, target_labels)) if i not in initial_indices], dtype='object')
        training_data, target_labels = remaining_data[:, 0], remaining_data[:, 1]
        
        epoch = 0
        while epoch < max_epochs:
            for i, sample in enumerate(training_data):
                distances = [np.sum((weight - sample) ** 2) for weight in self.weights]
                sorted_indices = np.argsort(distances)[:2]
                closest_index, second_closest_index = sorted_indices[0], sorted_indices[1]
                
                ratio = distances[closest_index] / distances[second_closest_index]
                
                if ratio <= epsilon:
                    if target_labels[i] == self.labels[closest_index] and target_labels[i] != self.labels[second_closest_index]:
                        self.weights[closest_index] += learning_rate * (sample - self.weights[closest_index])
                        self.weights[second_closest_index] -= learning_rate * (sample - self.weights[second_closest_index])

            learning_rate *= decay_factor
            epoch += 1


## LVQ 2.1

In [4]:
class LVQ2_1(LVQ2):
    def train(self, training_data, target_labels, learning_rate, decay_factor, max_epochs, epsilon=0.3):
        unique_labels, initial_indices = np.unique(target_labels, return_index=True)
        self.labels = unique_labels
        self.weights = training_data[initial_indices].astype(np.float64)
        remaining_data = np.array([sample for i, sample in enumerate(zip(training_data, target_labels)) if i not in initial_indices], dtype='object')
        training_samples, target_labels = remaining_data[:, 0], remaining_data[:, 1]
        epoch = 0

        while epoch < max_epochs:
            for i, sample in enumerate(training_samples):
                distances = [sum((weight - sample) ** 2) for weight in self.weights]
                distances = np.array(distances)
                closest_indices = np.argpartition(distances, 1)[:2]
                closest_distances = distances[closest_indices]
                sorted_closest_indices = closest_indices[np.argsort(closest_distances)]
                closest_index = sorted_closest_indices[0]
                second_closest_index = sorted_closest_indices[1] 

                if closest_distances[0] / closest_distances[1] >= (1 - epsilon):
                    if unique_labels[closest_index] != target_labels[i]:
                        if unique_labels[second_closest_index] == target_labels[i]:
                            self.weights[closest_index] -= learning_rate * (sample - self.weights[closest_index])
                            self.weights[second_closest_index] += learning_rate * (sample - self.weights[second_closest_index])
                    elif unique_labels[second_closest_index] == target_labels[i]:
                        self.weights[closest_index] += learning_rate * (sample - self.weights[closest_index])
                        self.weights[second_closest_index] += learning_rate * (sample - self.weights[second_closest_index])

            learning_rate *= decay_factor
            epoch += 1


# Percobaan

Lakukan training pada setiap model LVQ dengan parameter :
- Learning rate 0.5
- Decay factor 0.8
- Epoch maksimum 100
- Epsilon 0.3

Hitung akurasi dari LVQ dengan calc_accuracy

In [5]:
def calculate_accuracy(predicted_labels, true_labels):
    correct_predictions = [1 if predicted_labels[i] == true_labels[i] else 0 for i in range(len(predicted_labels))]
    return sum(correct_predictions) / len(predicted_labels)

Gunakan data berikut untuk melakukan percobaan

In [6]:
X, y = make_classification(n_samples=100, n_features=2, n_redundant=0, n_informative=2, n_classes=4, n_clusters_per_class=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [7]:
learning_rate = 0.5
decay_factor = 0.8
max_epochs = 100
epsilon = 0.3

# Train and test LVQ1
lvq1 = LVQ()
lvq1.train(X_train, y_train, learning_rate, decay_factor, max_epochs)
predictions_lvq1 = lvq1.predict(X_test)
accuracy_lvq1 = calculate_accuracy(predictions_lvq1, y_test)
print(f'Accuracy LVQ1: {accuracy_lvq1}')

# Train and test LVQ2
lvq2 = LVQ2()
lvq2.train(X_train, y_train, learning_rate, decay_factor, max_epochs, epsilon=epsilon)
predictions_lvq2 = lvq2.predict(X_test)
accuracy_lvq2 = calculate_accuracy(predictions_lvq2, y_test)
print(f'Accuracy LVQ2: {accuracy_lvq2}')

# Train and test LVQ2.1
lvq2_1 = LVQ2_1()
lvq2_1.train(X_train, y_train, learning_rate, decay_factor, max_epochs, epsilon=epsilon)
predictions_lvq2_1 = lvq2_1.predict(X_test)
accuracy_lvq2_1 = calculate_accuracy(predictions_lvq2_1, y_test)
print(f'Accuracy LVQ2.1: {accuracy_lvq2_1}')

Weights: [[-1.61874379 -1.60093655]
 [ 2.25829101 -2.17176077]
 [-0.70645229  1.24793325]
 [ 1.01281198  1.61813049]]
Weights: [[-1.56084668 -1.53909152]
 [ 2.12258451 -2.07468848]
 [-0.80797913  1.37101327]
 [ 1.02031097  1.66240381]]
Weights: [[-1.50894062 -1.49345983]
 [ 1.99801684 -1.99375503]
 [-0.88206437  1.41432414]
 [ 0.93648713  1.57141144]]
Weights: [[-1.45288891 -1.43326479]
 [ 1.81212519 -1.84493875]
 [-0.97561059  1.4241893 ]
 [ 0.9782837   1.53190808]]
Weights: [[-1.42063506 -1.40948586]
 [ 1.73316469 -1.80412195]
 [-1.05835885  1.41832503]
 [ 1.03118436  1.50775007]]
Weights: [[-1.37022253 -1.42422562]
 [ 1.65063546 -1.71293995]
 [-1.3367094   1.50493016]
 [ 1.51985538  1.6681956 ]]
Weights: [[-1.46924686 -1.45240524]
 [ 1.79587764 -1.8237649 ]
 [-1.22985502  1.47277602]
 [ 1.11694224  1.43053265]]
Weights: [[-1.48867198 -1.45714336]
 [ 1.76062925 -1.78062082]
 [-1.16870072  1.42110908]
 [ 1.09412378  1.34551348]]
Weights: [[-1.42820624 -1.32557996]
 [ 1.73694422 -1.760

# Analisis

1. Jika terdapat perbedaan performa pada setiap model LVQ, mengapa hal itu dapat terjadi?

Perbedaan performa antara model LVQ1, LVQ2, dan LVQ2.1 disebabkan oleh perbedaan algoritma pembaruan bobot serta kondisi pembelajaran yang diterapkan pada masing-masing model. LVQ1 menggunakan pendekatan dasar dengan memperbarui bobot hanya berdasarkan hubungan antara sampel data dengan vektor bobot terdekatnya, tanpa mempertimbangkan konteks yang lebih kompleks seperti rasio jarak atau hubungan antar kelas. Pada algoritma LVQ2 memperkenalkan konsep rasio jarak (distance ratio) dengan parameter epsilon untuk menentukan apakah pembaruan bobot dilakukan, serta memperhitungkan vektor bobot kedua terdekat, sehingga mampu menangkap lebih banyak informasi mengenai distribusi data, terutama pada data yang saling tumpang tindih (overlapping). Pada LVQ2.1  menerapkan kondisi tambahan terkait label target dan rasio jarak untuk meningkatkan akurasi dalam skenario di mana data memiliki pola yang lebih kompleks. Perbedaan ini membuat setiap model memiliki tingkat sensitivitas yang berbeda terhadap struktur data, distribusi kelas, dan tingkat noise, sehingga menghasilkan performa yang bervariasi tergantung pada karakteristik dataset yang digunakan.

2. Jelaskan resiko penggunaan Euclidean Distance sebagai pengukur jarak weight dengan komponen masukan vektor (X)

Penggunaan Euclidean Distance sebagai pengukur jarak antara bobot (weight) dengan komponen vektor masukan (X) dalam LVQ memiliki beberapa risiko. Salah satu risiko utama adalah sensitivitas Euclidean Distance terhadap skala data. Jika fitur dalam dataset memiliki skala yang berbeda, fitur dengan nilai lebih besar akan mendominasi perhitungan jarak, sehingga mengaburkan kontribusi fitur dengan nilai lebih kecil. Hal ini dapat menyebabkan model LVQ memberikan pembaruan bobot yang bias terhadap dimensi tertentu, mengurangi efektivitas proses pembelajaran. Selain itu, Euclidean Distance mengasumsikan bahwa semua fitur bersifat independen dan memiliki hubungan linier, yang tidak selalu sesuai dengan data dunia nyata. Pada dataset dengan hubungan non-linier antar fitur, pengukuran jarak ini dapat menghasilkan representasi yang kurang akurat. Risiko lainnya adalah sensitivitas terhadap outlier; nilai ekstrem dalam data dapat memperbesar jarak secara signifikan, sehingga mempengaruhi arah pembaruan bobot. Oleh karena itu, penting untuk memastikan bahwa data telah dinormalisasi dan relevansi fitur dievaluasi sebelum menggunakan Euclidean Distance dalam LVQ.